In [1]:
import tensorflow as tf

n_inputs = 3
n_neurons = 5

X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))

Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b)

init = tf.global_variables_initializer()


In [2]:
import numpy as np

X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]])
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]])

with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

In [3]:
print(Y0_val)

[[-0.79353786  0.86225009  0.41873506 -0.96197152 -0.99739718]
 [ 0.57355744  0.99893802  0.99935508  0.59780252 -1.        ]
 [ 0.98323286  0.99999243  0.99999958  0.99754673 -1.        ]
 [ 1.         -0.99999344  0.99463725  1.         -0.9994486 ]]


In [4]:
n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10

learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

logits = tf.layers.dense(states, n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)

loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

In [8]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/data/")
X_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels

Extracting /data/train-images-idx3-ubyte.gz
Extracting /data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /data/t10k-labels-idx1-ubyte.gz


In [9]:
n_epochs = 100
batch_size = 100

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

0 Train accuracy: 0.98 Test accuracy: 0.9371
1 Train accuracy: 0.95 Test accuracy: 0.9519
2 Train accuracy: 0.98 Test accuracy: 0.9571
3 Train accuracy: 0.98 Test accuracy: 0.9604
4 Train accuracy: 0.94 Test accuracy: 0.9665
5 Train accuracy: 0.98 Test accuracy: 0.9661
6 Train accuracy: 0.99 Test accuracy: 0.9634
7 Train accuracy: 0.99 Test accuracy: 0.9687
8 Train accuracy: 1.0 Test accuracy: 0.9691
9 Train accuracy: 0.98 Test accuracy: 0.9675
10 Train accuracy: 0.98 Test accuracy: 0.9658
11 Train accuracy: 0.97 Test accuracy: 0.9675
12 Train accuracy: 0.95 Test accuracy: 0.9743
13 Train accuracy: 0.99 Test accuracy: 0.9702
14 Train accuracy: 0.98 Test accuracy: 0.9702
15 Train accuracy: 0.99 Test accuracy: 0.9733
16 Train accuracy: 0.98 Test accuracy: 0.9763
17 Train accuracy: 0.98 Test accuracy: 0.9692
18 Train accuracy: 0.96 Test accuracy: 0.9627
19 Train accuracy: 0.98 Test accuracy: 0.9656
20 Train accuracy: 0.99 Test accuracy: 0.974
21 Train accuracy: 0.98 Test accuracy: 0.9684


In [21]:
tf.reset_default_graph()
n_steps = 20
n_inputs = 1
n_neurons = 100
n_outputs = 1

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])
# cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu)
cell = tf.contrib.rnn.OutputProjectionWrapper(
    tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu),
    output_size=n_outputs
)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

In [22]:
learning_rate = 0.001
loss = tf.reduce_mean(tf.square(outputs-y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()